# TalentScout – Intelligent Hiring Assistant Chatbot

This project is developed as part of the AI/ML Intern assignment.
The chatbot performs initial candidate screening by:
- Collecting candidate information
- Understanding the candidate’s tech stack
- Generating technical interview questions using an LLM
- Maintaining conversational context
- Ending the conversation gracefully

All data used is simulated and GDPR-safe.


Install & Import Libraries

In [ ]:
!pip install openai textblob


In [ ]:
from openai import OpenAI
from textblob import TextBlob

In [ ]:
import openai
# IMPORTANT: Replace 'YOUR_OPENAI_API_KEY' with your actual OpenAI API key.
# For better security, consider using Colab's secrets management feature.
openai.api_key = "sk-----d54"

Prompt Engineering

In [ ]:
INFO_PROMPT = """
You are TalentScout, a professional hiring assistant chatbot.
Collect the following details from the candidate one by one:
- Full Name
- Email Address
- Phone Number
- Years of Experience
- Desired Position(s)
- Current Location
- Tech Stack

Ask only ONE question at a time.
Stay professional and concise.
"""

TECH_QUESTION_PROMPT = """
You are a technical interviewer.
Based on the following tech stack:
{tech_stack}

Generate 3 to 5 technical interview questions per technology.
Do NOT include answers.
Questions should range from basic to intermediate level.
"""

FALLBACK_PROMPT = """
The user input is unclear.
Politely ask the candidate to rephrase without leaving the hiring context.
"""


Sentiment Analysis

In [ ]:
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    return "Neutral"


Conversation Logic (Context Handling)

In [ ]:
questions = [
    "What is your full name?",
    "What is your email address?",
    "What is your phone number?",
    "How many years of experience do you have?",
    "Which position are you applying for?",
    "What is your current location?",
    "Please list your tech stack (languages, frameworks, tools)."
]

candidate_data = {}
current_step = 0


Chatbot Function (Core Requirement)

In [ ]:
def hiring_assistant(user_input):
    global current_step

    exit_keywords = ["exit", "quit", "bye", "end"]

    if user_input.lower() in exit_keywords:
        return "Thank you for your time! Our recruitment team will contact you soon."

    sentiment = get_sentiment(user_input)

    if current_step < len(questions):
        candidate_data[questions[current_step]] = user_input
        current_step += 1
        return questions[current_step] if current_step < len(questions) else generate_tech_questions()

    return fallback_response()


LLM-Based Technical Question Generation

In [ ]:
def generate_tech_questions():
    tech_stack = candidate_data["Please list your tech stack (languages, frameworks, tools)."]

    prompt = TECH_QUESTION_PROMPT.format(tech_stack=tech_stack)

    client = OpenAI(api_key=openai.api_key)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

Fallback Handling

In [ ]:
def fallback_response():
    return "I didn’t quite understand that. Could you please rephrase your response?"


 Start the Chatbot (User Interaction)

In [ ]:
# Reset conversation state for a fresh start
current_step = 0
candidate_data = {}

print("🤖 TalentScout Hiring Assistant")
print("Type 'exit' anytime to end the conversation.\n")

print("Hello! I’m TalentScout, your AI Hiring Assistant.")
print(questions[current_step])

while True:
    user_input = input("You: ")
    response = hiring_assistant(user_input)
    print("Bot:", response)

    if "Thank you for your time" in response:
        break

🤖 TalentScout Hiring Assistant
Type 'exit' anytime to end the conversation.

Hello! I’m TalentScout, your AI Hiring Assistant.
What is your full name?
You: Sakshi Patel
Bot: What is your email address?
You: sakshi@gmail.com'
Bot: What is your phone number?
You: exit
Bot: Thank you for your time! Our recruitment team will contact you soon.
